In [1]:
!pip install geopy
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 3.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import geopy.distance
event_data = pd.read_csv("meetup_event_sample.csv")
group_data = pd.read_csv("meetup_group_sample.csv")

radius = 100 ##change radius according to your requirement

event_data.columns,group_data.columns
len(event_data),len(group_data)

total_events = len(event_data)

In [24]:
# Drop unwanted columns from group_data and select relevant columns from event_data
temp1 = group_data.drop(['Unnamed: 0','name'], axis=1)
temp2 = event_data[['group_id', 'venue_lat', 'venue_lon']]

# Merge the two dataframes on 'group_id'
temp3 = pd.merge(temp1, temp2, on='group_id')

# Drop the 'group_id' column since it is no longer needed
df = temp3.drop('group_id',axis=1)

# Convert 'category_name' column to categorical type and encode it as a numeric code
df['category_name'] = df['category_name'].astype('category')
df['category_code'] = df['category_name'].cat.codes

# Drop duplicate rows based on 'category_name' and 'category_code'
unique_pairs = df.drop_duplicates(subset=['category_name', 'category_code'])

# Sort the unique pairs by 'category_code'
sorted_df = unique_pairs.sort_values(by='category_code')

# Drop the 'venue_lat' and 'venue_lon' columns since they are no longer needed
sorted_df = sorted_df.drop(['venue_lat', 'venue_lon'],axis=1)

# Drop the 'category_name' column from df since it is no longer needed
df = df.drop('category_name',axis=1)

# Shuffle the rows of df, select the first 500 rows, and assign to shuffled_df
shuffled_df = df.sample(frac=1).head(500)

# Create a dictionary with 'category_code' as keys and 'category_name' as values
my_dict = sorted_df.set_index('category_code')['category_name'].to_dict()



***Convert dataframe into numpy to make multiplications easy***

In [25]:
import numpy as np
##convert dataframe to numpy
num = df.to_numpy()


In [26]:
import numpy as np
import geopy.distance

def calc(category, merged_data, radius=1000):
  # Initialize variables
  N = 0
  total_N = len(merged_data)
  repeated_total = np.zeros(30)
  arr = np.zeros(30)
  main_arr = np.zeros(30)
  fin_arr = np.zeros(30)

  # Loop through merged_data to calculate distances and counts
  for i in merged_data:
    # Update repeated_total array with count of each category
    repeated_total[int(i[2])] += 1
    if i[2] == category:
      # If category matches category, calculate distances and counts
      for j in merged_data:
        e1 = (i[0], i[1])
        e2 = (j[0], j[1])
        dist = geopy.distance.geodesic(e1, e2).m 
        if dist < radius:
          N += 1
          arr[int(j[2])] += 1

      # Calculate main_arr
      for k in range(len(arr)):
        val = arr[category] / (N - arr[k])
        main_arr[k] += val

  # Calculate fin_arr
  for i in range(30):
    if repeated_total[i] != 0:
      fin_arr[i] = (total_N - repeated_total[category])/(repeated_total[category] * repeated_total[i])

  # Calculate scores_arr
  scores_arr = np.zeros(30)
  for i in range(30):
    scores_arr[i] = main_arr[i] * fin_arr[i] 

  return scores_arr


In [27]:
all_scores = []

for i in range(30):
  ## for every value get the attrativeness of the corrosponding values 
  all_scores.append(calc(i,num))



<ipython-input-26-7bf853b82771>:29: RuntimeWarning: divide by zero encountered in double_scalars
  val = arr[category] / (N - arr[k])
<ipython-input-26-7bf853b82771>:35: RuntimeWarning: divide by zero encountered in double_scalars
  fin_arr[i] = (total_N - repeated_total[category])/(repeated_total[category] * repeated_total[i])
<ipython-input-26-7bf853b82771>:40: RuntimeWarning: invalid value encountered in double_scalars
  scores_arr[i] = main_arr[i] * fin_arr[i]


In [28]:
# Vertically stack all elements in the list 'all_scores' to create a 2D NumPy array
array_of_arrays = np.vstack(all_scores)


In [29]:
# Create a Pandas DataFrame from the NumPy array 'array_of_arrays'
df = pd.DataFrame(array_of_arrays)
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1.518034,4.399436,0.630122,5.857539,5.853370,1.058230,2.555657,8.830624,8.780054,0.963599,...,17.560109,5.881428,1.261593,0.839309,0.180011,2.197526,17.575194,0.397451,0.0,0.0
1,2.925940,9.288024,1.218122,11.703761,11.513945,2.031873,5.015897,17.270917,17.270917,1.859239,...,34.541834,11.513945,2.564888,1.688732,0.347622,4.317729,34.541834,0.756329,0.0,0.0
2,2.746635,8.125094,1.204801,10.833458,10.870945,2.022909,4.708757,16.270280,16.248441,1.817667,...,32.496882,10.899870,2.338847,1.557780,0.335155,4.076511,32.660873,0.715076,0.0,0.0
3,1.649150,5.154907,0.682407,6.760827,6.466573,1.141160,2.827115,9.699860,9.699860,1.041569,...,19.399720,6.466573,1.413557,0.967886,0.194529,2.499859,19.399720,0.450214,0.0,0.0
4,2.472816,7.418448,1.162312,9.891265,10.703314,1.938836,4.239113,14.836897,14.836897,1.593551,...,29.673794,9.891265,2.162677,1.376249,0.292564,3.709224,30.277478,0.618126,0.0,0.0
5,3.155564,9.387760,1.374347,12.517014,12.566542,2.396798,5.440882,18.802082,18.775521,2.127077,...,37.551041,12.594501,2.693844,1.811644,0.388175,4.708976,37.699627,0.793023,0.0,0.0
6,0.783881,2.304982,0.330392,3.056099,3.047602,0.564883,1.331020,4.591448,4.571403,0.506669,...,9.142807,3.061955,0.658615,0.439140,0.092055,1.144333,9.158316,0.202720,0.0,0.0
7,21.028523,63.063725,8.700690,84.084967,84.084967,14.844992,36.042573,inf,126.127451,13.286025,...,252.254902,84.084967,18.019737,11.472171,1.708695,31.531863,252.254902,5.052701,0.0,0.0
8,27.666667,83.000000,11.448276,110.666667,110.666667,19.529412,47.428571,166.000000,498.000000,17.473684,...,332.000000,110.666667,23.714286,22.636364,2.228188,41.500000,332.000000,6.640000,0.0,0.0
9,2.946106,8.694919,1.253425,11.593225,11.592262,2.176730,5.033988,17.450643,17.387198,1.973023,...,34.774397,11.684644,2.489882,1.667812,0.358619,4.355490,34.778337,0.748487,0.0,0.0


In [30]:
##delete last col
df = df.drop(df.columns[-1], axis=1)


In [31]:
##delete last row
df = df.drop(df.index[-1], axis=0)


In [32]:
df = df.rename(columns=my_dict)
new_col = pd.Series(my_dict)

In [33]:
df.insert(loc=0, column='new_col', value=new_col)

In [34]:
df

,new_col,Arts & Culture,Book Clubs,Career & Business,Cars & Motorcycles,Community & Environment,Dancing,Education & Learning,Fashion & Beauty,Fitness,...,Outdoors & Adventure,Pets & Animals,Photography,Religion & Beliefs,Singles,Socializing,Sports & Recreation,Support,Tech,Writing
0,Arts & Culture,1.518034,4.399436,0.630122,5.857539,5.853370,1.058230,2.555657,8.830624,8.780054,...,1.128673,17.560109,5.881428,1.261593,0.839309,0.180011,2.197526,17.575194,0.397451,0.0
1,Book Clubs,2.925940,9.288024,1.218122,11.703761,11.513945,2.031873,5.015897,17.270917,17.270917,...,2.158865,34.541834,11.513945,2.564888,1.688732,0.347622,4.317729,34.541834,0.756329,0.0
2,Career & Business,2.746635,8.125094,1.204801,10.833458,10.870945,2.022909,4.708757,16.270280,16.248441,...,2.043837,32.496882,10.899870,2.338847,1.557780,0.335155,4.076511,32.660873,0.715076,0.0
3,Cars & Motorcycles,1.649150,5.154907,0.682407,6.760827,6.466573,1.141160,2.827115,9.699860,9.699860,...,1.212483,19.399720,6.466573,1.413557,0.967886,0.194529,2.499859,19.399720,0.450214,0.0
4,Community & Environment,2.472816,7.418448,1.162312,9.891265,10.703314,1.938836,4.239113,14.836897,14.836897,...,1.947544,29.673794,9.891265,2.162677,1.376249,0.292564,3.709224,30.277478,0.618126,0.0
5,Dancing,3.155564,9.387760,1.374347,12.517014,12.566542,2.396798,5.440882,18.802082,18.775521,...,2.358891,37.551041,12.594501,2.693844,1.811644,0.388175,4.708976,37.699627,0.793023,0.0
6,Education & Learning,0.783881,2.304982,0.330392,3.056099,3.047602,0.564883,1.331020,4.591448,4.571403,...,0.579023,9.142807,3.061955,0.658615,0.439140,0.092055,1.144333,9.158316,0.202720,0.0
7,Fashion & Beauty,21.028523,63.063725,8.700690,84.084967,84.084967,14.844992,36.042573,inf,126.127451,...,15.770000,252.254902,84.084967,18.019737,11.472171,1.708695,31.531863,252.254902,5.052701,0.0
8,Fitness,27.666667,83.000000,11.448276,110.666667,110.666667,19.529412,47.428571,166.000000,498.000000,...,20.750000,332.000000,110.666667,23.714286,22.636364,2.228188,41.500000,332.000000,6.640000,0.0
9,Food & Drink,2.946106,8.694919,1.253425,11.593225,11.592262,2.176730,5.033988,17.450643,17.387198,...,2.200578,34.774397,11.684644,2.489882,1.667812,0.358619,4.355490,34.778337,0.748487,0.0
